In [133]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 

In [7]:
conn = sqlite3.connect(r'C:\Users\shale\Downloads\archive (5)\database.sqlite') 

In [8]:
conn

In [37]:
data1 = pd.read_sql_query('select * from Reviews',conn)

In [40]:
data1.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [39]:
data = pd.read_sql_query('select * from Reviews where Score!=3',conn)

In [41]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [42]:
data2 = pd.read_sql_query('select * from Reviews where Score=3',conn)

In [43]:
data2.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,46,B001EO5QW8,A39Z97950MCTQE,K. A. Freel,0,0,3,1205193600,Hearty Oatmeal,This seems a little more wholesome than some o...
1,48,B001EO5QW8,A26AY1TFK8BQXQ,"kbogo ""shoelover""",1,2,3,1200096000,Mushy,"The flavors are good. However, I do not see a..."
2,50,B001EO5QW8,A276999Y6VRSCQ,JMay,0,1,3,1334016000,Same stuff,This is the same stuff you can buy at the big ...
3,54,B000G6RPMY,AQ9DWWYP2KJCQ,"Roel Trevino ""protomex""",0,0,3,1278028800,not ass kickin,we're used to spicy foods down here in south t...
4,61,B004N5KULM,A1ZR8O62VSU4OK,"Lisa J. Szlosek ""lisa""",2,4,3,1318723200,Better price for this at Target,Watch your prices with this. While the assort...


In [50]:
def partition(x):
    if x<3:
        return 'negative'
    return 'postive'

In [51]:
actual_score = data['Score']
positivenegative = actual_score.map(partition)
data['Score'] = positivenegative

In [52]:
data['score'].head()

0     postive
1    negative
2     postive
3    negative
4     postive
Name: score, dtype: object

In [53]:
data.shape

(525814, 11)

In [54]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,postive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,postive
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,negative
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,postive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,postive
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,negative
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,postive,1350777600,Great taffy,Great taffy at a great price. There was a wid...,postive


In [55]:
i = 0
for sent in data['Text'].values:
    if(len(re.findall('<.*?',sent))):
        print(i)
        print(sent)
        break;
i+=1

0
I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [58]:
x=data['ProductId']

In [59]:
x.ravel()

array(['B001E4KFG0', 'B00813GRG4', 'B000LQOCH0', ..., 'B004I613EE',
       'B004I613EE', 'B001LR2CU2'], dtype=object)

In [65]:
data.loc[0:2,'Score']

0     postive
1    negative
2     postive
Name: Score, dtype: object

In [72]:
display =pd.read_sql_query('select * from Reviews where Score!=3 AND UserId="AR5J8UI46CURR" order by ProductId',conn)

In [73]:
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [84]:
sorted_data=data.sort_values('ProductId',axis=0,
    ascending=True,
    inplace=False)

In [85]:
sorted_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,postive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,postive
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,postive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",postive
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,postive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,postive
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,postive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,postive
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,postive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,postive


In [90]:
final=sorted_data.drop_duplicates(subset={"UserId","Text","ProfileName","Time"},keep='first',inplace=False)

In [91]:
final.shape


(364173, 11)

In [94]:
(final.size*1.0)/(data.size*1.0)*100

69.25890143662969

In [95]:
display1 = pd.read_sql_query("select * from Reviews where Score !=3 AND Id=44737 or Id=6422 order by ProductId",conn)

In [96]:
display1

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,6422,B001UOY7X6,A1UHSQI6HLMUJA,Cooker,0,0,5,1287273600,Amazing cooking chocolate,This is amazing chocolate for most baking. Man...


In [99]:
final=final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [100]:
print(final)

            Id   ProductId          UserId                  ProfileName  \
138706  150524  0006641040   ACITT7DI6IDDL              shari zychinski   
138688  150506  0006641040  A2IW4PEEKO2R0U                        Tracy   
138689  150507  0006641040  A1S4A3IQ2MU7V4        sally sue "sally sue"   
138690  150508  0006641040     AZGXZ2UUK6X  Catherine Hallberg "(Kate)"   
138691  150509  0006641040  A3CMRKGE0P909G                       Teresa   
...        ...         ...             ...                          ...   
178145  193174  B009RSR8HO   A4P6AN2L435PV                       romarc   
173675  188389  B009SF0TN6  A1L0GWGRK4BYPT                Bety Robinson   
204727  221795  B009SR4OQ2  A32A6X5KCP7ARG                      sicamar   
5259      5703  B009WSNWC4   AMP7K1O84DH1T                         ESTY   
302474  327601  B009WVB40S  A3ME78KVX31T21                         K'la   

        HelpfulnessNumerator  HelpfulnessDenominator    Score        Time  \
138706                

In [101]:
final.shape

(364171, 11)

In [103]:
final['Score'].value_counts()

postive     307061
negative     57110
Name: Score, dtype: int64

In [110]:
countvector = CountVectorizer()


In [113]:
finalcount = countvector.fit_transform(data['Text'].values)

In [114]:
finalcount.shape

(525814, 115282)

In [115]:
final['Text'].value_counts()

Diamond Almonds<br />Almonds are a good source of magnesium. One ounce contain approximately 20% of the RDA for an adult recommended dietary allowance (RDA)<br />Recommended Dietary Allowance (RDA) the amounts of selected nutrients considered adequate to meet the known nutrient needs of healthy people. The RDA is based on scientific knowledge and has been presented by a committee of the Food and Nutrition Board (FNB) of the National Academy of Sciences (NAS). The Canadian equivalent is the Recommended Nutrient Intakes. RDA is generally accepted throughout the world as a valid source of information. At least 40 different nations have as well as organizations have published standards similar to the RDA.<br /><br />Magnesium is the fourth most abundant mineral in the body and is essential to good health. Approximately 50% of total body magnesium is found in bone. The other half is found predominantly inside cells of body tissues and organs. Only 1% of magnesium is found in blood, but the 

In [116]:
finalcount.shape

(525814, 115282)

In [117]:
count=TfidfVectorizer()

In [118]:
finalcount1=count.fit_transform(data['Text'].values)

In [122]:
finalcount1.shape


(525814, 115282)

In [131]:
i=0;
for sent in data['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i+=1       

10
I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [136]:
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shale\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [145]:
sno = nltk.stem.SnowballStemmer('english')

In [160]:
def cleanhtml(sentence):
    cleanr=re.compile('<.*?')
    cleanedtext = re.sub(cleanr,'',sentence)
    return cleanedtext


In [177]:
def cleanpunc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|(|)|\|/]',r'' ,cleaned)
    return cleaned
print(stop)

{'if', "haven't", 'mustn', 'your', 'to', 'weren', 's', 'just', "wouldn't", 'a', "it's", "you've", 'from', 'against', 'its', 'so', 'how', 'until', 'has', 'most', 'after', 'ours', 'had', 'my', 'down', 'on', 'each', 'wasn', 'those', 'or', 'm', 'between', 'myself', "should've", 'when', 'have', 'been', 'o', 'before', 'by', 'these', "wasn't", 'their', 'not', 'will', 'other', 'an', 'y', 'nor', "she's", 'this', 'up', 'yourself', "doesn't", "won't", 'are', 'he', "needn't", 'am', 'them', 'too', 'yours', 'that', 'they', 'then', 'being', 'you', 'more', 'wouldn', 'don', 'while', 'couldn', 'but', "didn't", "shan't", 'below', 'few', 'the', 'once', 'why', 'himself', "weren't", "hasn't", 'again', 'll', 'him', 'which', "hadn't", 'shouldn', 'ourselves', "that'll", "mustn't", 'do', 'can', 'doesn', 'hasn', 'aren', 'isn', 'here', 'didn', 'who', 'what', 'about', 'ain', 'were', 'itself', 'd', "you're", 'any', 'her', 'me', 'because', 'does', 'both', 'in', 'above', 'now', 'and', 'theirs', 'where', 'such', 'we',

In [178]:
print(sno.stem('tasty'))

tasti


In [180]:
i=0
str1=''
all_positive_words=[]
all_negative_words=[]
final_string=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final['Score'].values)[i]=='positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i]=='negative' :
                        all_negative_words.append(s)
                else:
                    continue;
            else:
                continue;
    str1=b" ".join(filtered_sentence)        
    final_string.append(str1)
    i+=1    

In [183]:
final_string

[b'witti littl book make son laugh loud recit car drive along alway sing refrain hes learn whale india droop love new word book introduc silli classic book will bet son still abl recit memori colleg',
 b'grew read sendak book watch realli rosi movi incorpor love son love howev miss hard cover version paperback seem kind flimsi take two hand keep page open',
 b'fun way children learn month year learn poem throughout school year like handmot invent poem',
 b'great littl book read nice rhythm well good repetit littl one like line chicken soup rice child get month year wonder place like bombay nile eat well know get eat kid mauric sendak version ice skate treat rose head long time wont even know came surpris came littl witti book',
 b'book poetri month year goe month cute littl poem along love book realli fun way learn month poem creativ author purpos write book give children fun way learn month children also learn thing poetri rhythm read book',
 b'charm rhyme book describ circumst eat do

In [184]:
final['cleaned_text']=final_string

In [185]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score,cleaned_text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,postive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,postive,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,postive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",postive,b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,postive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,postive,b'fun way children learn month year learn poem...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,postive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,postive,b'great littl book read nice rhythm well good ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,postive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,postive,b'book poetri month year goe month cute littl ...
